# Exploratory Data Analysis of Powercreep in Pokemon

Original dataset taken from Josh Korngiebel on Kaggle.
https://www.kaggle.com/datasets/jkorn28/pokemon-stats-gens-1-9 

Dataset modified by me in the data prep folder. All operations in this notebook will be using the modified csv file.

## <p style="text-align: center;">Introduction</p>

### **Background**

Standing head and shoulders over every single media company and intellectual property group in the world is Pokemon. As the highest grossing media franchise of all time, Pokemon has cemented, no matter how big or small, a place in the minds of almost everyone. Because of the many games and multiplayer features added, communities of hobbyists dedicated to competitive Pokemon battling have developed. 

These hobbyists have developed remarkably optimized Pokemon teams, and continue to battle online with a thriving competitive scene. Everytime a new generation of Pokemon is released, the online competitive scene has to scramble to adjust to what has been deemed "powercreep". Powercreep is a term used throughout gaming to generally refer to the phenomenon of new > old, that is, new content released by the maker of the game more often than not tend to outclass older content.

### **Goal**

Leveraging Python libraries, we hope to both quantify and visualize the effect of powercreep. This will be done by analyzing what makes a Pokemon "strong", and using that as a metric to view powercreep over the 9 generations of Pokemon.

## <p style="text-align: center;">Table of Contents</p>

1. The Data
    - Loading
    - Previewing & Explaining
    - Error Testing
1. Analysis
    - Stat Distributions
    - Special Pokemon
        - Legendaries
        - Mythicals
        - Notables
        - "Optimized" Pokemon
        - Average Type Weakness
1. Conclusions
    

## <p style="text-align: center;">The Data</p>

### **Loading (+ Libraries)**

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
pokedf = pd.read_csv('pokemon_data_custom.csv')

The data has already been prepared in another folder on this repository. For more detailed information on what changes were made to the original dataset by Josh Korngiebel, refer to that folder. Briefly, useful columns were added for classification, and 7 missing Pokemon were added.

### **Previewing & Explaining**

In [7]:
pokedf.head()

,Dex No,Name,Base Name,Type 1,Type 2,BST,HP,Attack,Defense,Sp. Attack,Sp. Defense,Speed,Generation,Legendary,Mythical,Notable
0,1,Bulbasaur,Bulbasaur,GRASS,POISON,318,45,49,49,65,65,45,1,False,False,False
1,2,Ivysaur,Ivysaur,GRASS,POISON,405,60,62,63,80,80,60,1,False,False,False
2,3,Venusaur,Venusaur,GRASS,POISON,525,80,82,83,100,100,80,1,False,False,False
3,3,Mega Venusaur,Venusaur,GRASS,POISON,625,80,100,123,122,120,80,6,False,False,False
4,4,Charmander,Charmander,FIRE,-,309,39,52,43,60,50,65,1,False,False,False


In [8]:
pokedf.tail()

,Dex No,Name,Base Name,Type 1,Type 2,BST,HP,Attack,Defense,Sp. Attack,Sp. Defense,Speed,Generation,Legendary,Mythical,Notable
1196,1013,Sinistcha,Sinistcha,GRASS,GHOST,508,71,60,106,121,80,70,9,False,False,False
1197,1014,Okidogi,Okidogi,POISON,FIGHT,555,88,128,115,58,86,80,9,True,False,False
1198,1015,Munkidori,Munkidori,POISON,PSYCHIC,555,88,75,66,130,90,106,9,True,False,False
1199,1016,Fezandipiti,Fezandipiti,POISON,FAIRY,555,88,91,82,70,125,99,9,True,False,False
1200,1017,Ogerpon,Ogerpon,GRASS,-,550,80,120,84,60,96,110,9,True,False,False


The data contains 16 columns.
- Dex No: Refers to the official *Pokedex* number, of the Pokemon. Every Pokemon has a unique Pokedex number, or as it will be referred to, Dex number.

- Name: Self explanatory with one caveat, certain Pokemon are considered *alternate forms*, that is, they are the same species of Pokemon, but simply a different variant. One example is the Mega Venusaur visible in the dataframe head. Notice the entry shares the same Dex number as its base form, Venusaur.

- Base Name: Every Pokemon has the same Base Name as Name, except the aformentioned variants, who instead have the name of the original form.

- Type 1 and 2: There are 18 elemental "types" of Pokemon, which will be explained in more detail later. Notice that not all Pokemon have two types, some only have one. In that case, the Type 2 column will have a placeholder '-'